## Mixed Logit and Random Coefficients Estimation
Mixed logit allows for more flexible estimation. The decision makers can be treated to have heterogenous preferences. This is implemented via distributional assumptions on the model's parameters. For instance, the coefficients can be assumed to be normally distributed.

**Example:** Electricity demand example from the mlogit package (Kenneth Train and Yves Croissant). The dataset is from a survey/ hypothetical choice questionnaire.



In [6]:
library("mlogit")
data("Electricity", package="mlogit")
Electr <- mlogit.data(Electricity, id = "id", choice = "choice", varying = 3:26, shape = "wide", sep = "")

### Variables
The choice indicator _choice_ is one of 4 choices (e.g. 1,2,3,4). Explanatory variabels include a fixed price per unit (i.e cents/Kwh)  _pf_ which varies across suppliers, contract length _cf_ in years, an indicator for electricity supplier being local _lo_, another indicator if the supplier is well known company _wk_, and a (variable/dynamic price) variable _tod_ rate under which price is 11 cents/unit  between 8am-8pm and an off-peak price of 5 cents/Kwh from 8pm-8am. These tod dynamic prices were constant across suppliers (in the hypothetical experiments presented to each individual). There is also a seasonal rate _seas_ under which prices are 10 cents/Kwh in summer, 8 in winter, and 6 for fall/spring. These seasonal prices also do not vary across suppliers. 


**Note:** The estimation step below takes a couple of minutes to run.

In [4]:
Elec.mxl <- mlogit(choice~pf+cl+loc+wk+tod+seas|0,Electr, 
                   rpar=c(pf='n',cl='n',loc='n',wk='n',tod='n',seas='n'),#random parameters assumed dist. normally
                   R=100,halton=NA, print.level=0, 
                   panel = TRUE #panel data
                  )
summary(Elec.mxl)


Call:
mlogit(formula = choice ~ pf + cl + loc + wk + tod + seas | 0, 
    data = Electr, rpar = c(pf = "n", cl = "n", loc = "n", wk = "n", 
        tod = "n", seas = "n"), R = 100, halton = NA, panel = TRUE, 
    print.level = 0)

Frequencies of alternatives:
      1       2       3       4 
0.22702 0.26393 0.23816 0.27089 

bfgs method
21 iterations, 0h:1m:36s 
g'(-H)^-1g = 3.3E-07 
gradient close to zero 

Coefficients :
         Estimate Std. Error z-value  Pr(>|z|)    
pf      -0.973384   0.034324 -28.359 < 2.2e-16 ***
cl      -0.205557   0.013323 -15.428 < 2.2e-16 ***
loc      2.075733   0.080430  25.808 < 2.2e-16 ***
wk       1.475650   0.065168  22.644 < 2.2e-16 ***
tod     -9.052542   0.287219 -31.518 < 2.2e-16 ***
seas    -9.103772   0.289043 -31.496 < 2.2e-16 ***
sd.pf    0.219945   0.010840  20.291 < 2.2e-16 ***
sd.cl    0.378304   0.018489  20.461 < 2.2e-16 ***
sd.loc   1.482980   0.081305  18.240 < 2.2e-16 ***
sd.wk    1.000061   0.074182  13.481 < 2.2e-16 ***
sd.tod   2.

### Estimated willingness to pay
How much would the typical electricity customer be willing to pay to extend the contract duration by another year?

In [7]:
coef(Elec.mxl)['cl']/coef(Elec.mxl)['pf']

cl 
0.2111772

The typicaly customer dislikes having a longer contract, as indicated by negative sign for the coefficient on contract length _cl_  (the negative sign for price is to be expected.). So the typical customer is willing to pay an additional  0.21 _cents_ per KWh (from the ratio  -2055/-.9733) to _reduce_ contract length by 1 year. 

**Note:** Not all customers will behave the same way. The estimated coefficient for _cl_ is not negative for everyone! What is the size of the market segment who prefer **shorter** contracts? We can obtain an estimate based on the area under the normal distribution (to the left of 0) given the mean and standard deviation for _cl_.

In [8]:
pnorm(0-coef(Elec.mxl)['cl']/coef(Elec.mxl)['sd.cl']) ##standardize to unit normal, then get the cumulative prob.

cl 
0.70656

Thus, approximately 70% of customers prefer a short-term contracts and 30% prefer longer contracts. 

### Is the normal distibution too restrictive?
The estimated coefficient for price has a mean of -.97 and a standard deviation of 0.2199. Does this imply that an unreasonable fraction of the sample are estimated to have _positive_ price coefficients? 

In [9]:
pnorm(0-coef(Elec.mxl)['pf']/coef(Elec.mxl)['sd.pf'])

pf 
0.9999952

This indicates that virtually nobody (<.0001) is estimated to have positive price coefficients. A larger estimate would naturally cause some concern. In that case, we could drop the assumption. Some alternatives are to treat the coefficient for price as common (fixed) for all (by dropping it from _rpar_( ) in the _mlogit_( ) model), or to change the distribution from normal to something else, such as uniform or log-normal, both of which are built into the package. 

### Correlated parameters
The above example assumes that there is no correlation in across the parameters. This assumption can be dropped by using the option "correlation = TRUE" in _mlogit_( ). The estimation will naturally take much longer. 

In [10]:
Elec.mxl2 <- update(Elec.mxl, correlation=T)
summary(Elec.mxl2)


Call:
mlogit(formula = choice ~ pf + cl + loc + wk + tod + seas | 0, 
    data = Electr, rpar = c(pf = "n", cl = "n", loc = "n", wk = "n", 
        tod = "n", seas = "n"), R = 100, correlation = T, halton = NA, 
    panel = TRUE, print.level = 0)

Frequencies of alternatives:
      1       2       3       4 
0.22702 0.26393 0.23816 0.27089 

bfgs method
34 iterations, 0h:4m:9s 
g'(-H)^-1g = 0.364 
last step couldn't find higher value 

Coefficients :
            Estimate Std. Error  z-value  Pr(>|z|)    
pf        -0.8590278  0.0351144 -24.4637 < 2.2e-16 ***
cl        -0.2031318  0.0137964 -14.7236 < 2.2e-16 ***
loc        2.5703767  0.0924607  27.7997 < 2.2e-16 ***
wk         1.8659691  0.0759564  24.5663 < 2.2e-16 ***
tod       -8.3724950  0.3036069 -27.5768 < 2.2e-16 ***
seas      -8.6348459  0.3062401 -28.1963 < 2.2e-16 ***
pf.pf     -0.2368250  0.0414717  -5.7105 1.126e-08 ***
pf.cl      0.1720463  0.0165057  10.4235 < 2.2e-16 ***
pf.loc    -0.2269268  0.0943935  -2.4041 0.016214

### Is correlation a factor?
This can be tested various ways. The package offers options for LR, Wald, and Scoretest.

In [16]:
lrtest(Elec.mxl, Elec.mxl2)

#Df,LogLik,Df,Chisq,Pr(>Chisq)
12,-3952.488,NA,NA,NA
27,-3802.897,15,299.1819,8.267607e-55


The test rejects the hypothesis that the parameters are not correlated.